# MODFLOW 6: introducción.


## Resumen.

El propósito de este documento es describir la arquitectura de MODFLOW 6, sus componentes y paquetes, así como los elementos principales que se usan en una simulación de flujo con el modelo GWF. Esta descripción se basa en la documentación de MODFLOW 6 y solo se explican los conceptos principales para entender la forma en que interactúan todas las componentes y paquetes durante una simulación.

<p xmlns:cc="http://creativecommons.org/ns#" xmlns:dct="http://purl.org/dc/terms/"><a property="dct:title" rel="cc:attributionURL" href="https://github.com/luiggix/RTWMA/">MODFLOW 6: introducción</a> by <b>Luis M. de la Cruz Salas (2025)</b> is licensed under <a href="http://creativecommons.org/licenses/by-sa/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">Attribution-ShareAlike 4.0 International<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1"></a>.</p> 

# Introducción.
[MODFLOW 6](https://www.usgs.gov/software/modflow-6-usgs-modular-hydrologic-model) utiliza la Programación Orientada a Objetos (POO) para admitir una amplia gama de capacidades y para integrar las funcionalidades de todas las versiones y variantes de MODFLOW anteriores.

Esta última versión está centrada en la solución numérica de uno o más modelos hidrológicos numéricos que pueden conectarse e interactuar entre ellos. Cada modelo es un objeto de una clase que encapsula datos y métodos. Se pueden crear múltiples objetos (que representan modelos hidrológicos) en una sola simulación. 

Además, se utilizan los conceptos de abstracción y herencia para gestionar y ocultar la complejidad de los modelos, y para permitir que otros autores agreguen nuevos modelos de procesos hidrológicos, de forma sistemática y eficiente, sin interferir con otros desarrollos.

Este enfoque ha dado como resultado componentes de simulación genéricos, denominados **Modelos Numéricos**, **Intercambios Numéricos** y **Soluciones Numéricas**, diseñados para funcionar conjuntamente y abordar muchos de los requisitos de diseño de esta nueva versión. 

# Componentes.

En MODFLOW 6, una simulación consiste de una única ejecución, que puede incluir múltiples modelos de flujo (y de transporte). Estos modelos son configurados y controlados por componentes y paquetes. 

Una componente es un término general para describir una parte de MODFLOW 6, que puede ser un módulo, un objeto, una subrutina o un conjunto de estos. La figura 1 muestra las componentes principales que pueden ser usadas en una simulación de MODFLOW 6.

<figure>
  <img src="../figures/components_mf6.png" width=800px>
  <figcaption>Figura 1. Diagrama de componentes de MODFLOW 6. Tomada de [1].</figcaption>
</figure> 

Una simulación en MODFLOW 6 típicamente se basa en cuatro componentes, como se explica en [1]:
* **Models**. Un modelo numérico resuelve un proceso hidrológico; por ejemplo, el *GWF model*, resuelve numéricamente la ecuación de flujo subterráneo usando el método CVFD (*Control-Volume Finite-Difference*).
* **Exchange**. Un intercambio numérico facilita la comunicación entre dos modelos; por ejemplo, un intercambio GWF-GWF, permite que algunas celdas de un modelo GWF estén hidráulicamente conectadas con celdas del otro modelo GWF.
* **Solutions**. Una solución numérica resuelve uno o más modelos hidrológicos y utiliza métodos iterativos para resolver sistemas lineales y no-lineales.
* **Timing**. Este módulo controla el paso de tiempo y determina el fin de una simulación.


# Simulación de flujo con GWF.

En la figura 2 se muestra una representación de una simulación en MODFLOW 6 que contiene un solo modelo de GWF.

<figure>
  <img src="../figures/modflow00.png" width=300px hspace="5" vspace="5"/>
  <figcaption>Figura 2. Diagrama que muestra las componentes que se usan para la simulación de un solo modelo GWF. Las componentes son: (1) una simulación (que es el programa principal), (2) una componente para gestionar el tiempo, (3) una componente para gestionar la solución numérica y (4) un único modelo GWF. Figura tomada de [2].
  </figcaption>
</figure> 

## La simulación.

Una simulación consiste de una ejecución que puede incluir varios modelos (de flujo, de transporte, de intercambios, numéricos, etcétera). La simulación es la componente de mayor nivel y está controlada por el programa principal. La siguiente figura, tomada de [1], esquematiza el flujo de una simulación.

<img src="../figures/modflow01.png" width=200px hspace="5" vspace="5" style="float: left;"/>

El programa principal ejecuta varios procedimientos, conocidos como ***primary procedures*** en la secuencia apropiada para realizar y controlar la simulación.

* CR: Crea objetos de tipo *model*, *package*, *exchange* y/o *solution*.
* DF: Define algunos atributos de los objetos, por ejemplo el tamaño de algunos arreglos.
* AR: Genera los arreglos. También lee información que será constante durante toda la simulación.
* TU: Incrementa las variables del tiempo y calcula la longitudes del paso de tiempo.
* RP: Lee la información de los archivos de entrada, conforme se necesite, para actualizar los estreses hidrológicos u otras entradas que varían con el tiempo.
* CA: Actualiza las variables dependientes. Para soluciones numéricas, utiliza métodos iterativos para resolver sistemas de ecuaciones no lineales.
* OT: Escribe los resultados de la simulación en archivos para cada paso de tiempo o como sea requerido.
* FP: Escribe mensajes de terminación y cierra todos los archivos.
* DA: Desaloja toda la memoria.

Como puede observarse TU, RP, CA y OT se ejecutan varias veces dentro del ciclo temporal.
Cada paso en el diagrama representa ejecuciones para todos los *models*, *exchanges* y *solutions* que forman parte de la simulación. 

## La discretización temporal. 
Una simulación se divide en intervalos de tiempo, conocidos como periodos de estrés, **durante los cuales los datos de entrada de todos los estreses externos permanecen constantes**. Los periodos de estrés se dividen a su vez en pasos de tiempo, como se muestra en la siguiente figura (véase **Timing Module** en [1], pp 10-12):

<img src="../figures/time_step.png" width=500px>

Esta componente implementa y ejecuta las operaciones del procedimiento primario *Time Update* (TU).

La tabla siguiente describe algunos de los parámetros que se usan en este módulo:

|Variable|Tipo|Significado|
|--|--|:--|
|`PERLEN` | double| es la longitud del periodo de estrés| 
|`NSTP` |integer| es el número de pasos de tiempo en un periodo de estrés|
|`TSMULT` |double| es el multiplicador para calcular la longitud de pasos de tiempo sucesivos|


Para calcular la longitud del primer paso de tiempo en un periodo de estrés se hace lo siguiente:

* Para TSMULT $= 1$:

$$
\Delta t_1= \frac{\text{PERLEN}}{\text{NSTP}} 
$$

* Para TSMULT $\neq 1$:

$$
\Delta t_1= \text{PERLEN} \frac{\text{TSMULT} - 1}{\text{TSMULT}^\text{nstp}-1}
$$

La longitud de cada paso de tiempo sucesivo es calculada multiplicando la longitud del paso de tiempo previo por `TSMULT`.

$$
\Delta t = \Delta t_{old} \text{TSMULT}
$$


<div class="alert alert-block alert-info">
    
### Ejemplo 1. Periodo de estrés.

Considérese un modelo de flujo GWF que usa los paquetes de *River* y *Well*, en donde el tiempo de simulación es de $90$ días, con pasos de tiempo de un día. 

* Si el nivel del río o el bombeo de los pozos solo cambian cada $30$ días, se pueden usar $3$ períodos de estrés, cada uno de $30$ días. 

* De igual manera, si el bombeo o el nivel del río varían cada $3$ días, se usarían $30$ períodos de estrés de $3$ días.

* Cuando comienza un nuevo período de estrés, se deben redefinir todos los datos de estrés; sin embargo, la mayoría de los paquetes podrían reutilizar los datos del período de estrés anterior.

* En este ejemplo, se podrían reutilizar los datos del río en un nuevo período de estrés cuando solo cambian las tasas de bombeo mientras que el nivel del río permanece igual.

**NOTA.** Los períodos de estrés se implementan solo por conveniencia. Los paquetes que definen estreses dependientes del tiempo leen los datos de entrada en cada período de estrés. Los períodos de estrés facilitan la gestión de datos de entrada que se mantienen constantes por múltiples intervalos de tiempo. Sin embargo, no son inusuales las situaciones en las que surge la necesidad de cambiar los datos de estrés en cada paso de tiempo. En este caso, cada período de estrés debería constar de un solo paso de tiempo; como alternativa, se puede utilizar la funcionalidad de series temporales (*Time Series*) que está integrada en MODFLOW 6, véase **Utilities - Time Series** en [1], pp 27-28.

</div>

## La solución numérica.

La componente *Numerical Solution* es una parte clave del simulador, ya que se encarga de configurar el solucionador que se usará para resolver las ecuaciones de uno o más modelos y de los intercambios que los conectan.

En términos simples en esta componente se definen:

* el tipo de solucionador numérico que se usará (ej. Newton-Raphson, métodos iterativos lineales (CG o BiCGSTAB),
* los criterios de convergencia (número máximo de iteraciones, tolerancias, etc.),
* la asociación entre modelos y la solución numérica (por ejemplo, un modelo de flujo puede estar vinculado con una o varias soluciones numéricas en caso de que haya múltiples modelos o dominios acoplados).

El modelo numérico de GWF formula un sistema lineal de ecuaciones de la forma:

$$
\mathbf{A} \mathbf{x} = \mathbf{b}
$$

donde $\mathbf{A}$ es la matriz de coeficientes, $\mathbf{x}$ es el vector de variables dependientes (la carga hidráulica, por ejemplo) y $\mathbf{b}$ es el lado derecho del sistema con valores conocidos (ej. las condiciones iniciales y de frontera).

<img src="../figures/modflow02.png" width=450px hspace="5" vspace="5" style="float: left;"/>

* En la figura de la izquierda, tomada de [1], se muestra un ejemplo ilustrativo de una matriz para tres modelos numéricos, con sus respectivos intercambios que los conectan entre sí, que se resuelve por un solo componente de tipo *Numerical Solution*.
* La componente *Numerical Solution* resuelve el sistema de ecuaciones para uno o más modelos usando métodos iterativos.
  
* Estos métodos son capaces de manejar matrices dispersas, simétricas y no simétricas, de tal manera que es posible resolver problemas en mallas no estructuradas, formulaciones tipo Newton-Raphson, flujo de agua anisotrópico, transporte de solutos dispersivo, entre otros.

**Jerarquía de clases de los solucionadores numéricos.**

<img src="../figures/modflow_class_1.png" width=200px hspace="5" vspace="5" style="float: left;"/>

* *BaseSolutionType* es la clase base general para cualquier tipo de solución en MODFLOW 6; de esta superclase deben heredar todas las clases para calcular las soluciones numéricas.
  
* *NumericalSolutionType* Es una especialización de *BaseSolutionType*, que está orientada exclusivamente a métodos numéricos iterativos. Está diseñada para resolver uno o más modelos numéricos.

* *IterativeModelSolutionType* (IMS) es una implementación concreta de la clase *NumericalSolutionType* y se encarga de resolver uno o varios modelos acoplados. En una simulación se crean objetos de esta clase para calcular la solución numérica.

<img src="../figures/modflow03.png" width=350px hspace="5" vspace="5" style="float: right;"/>

* Los procedimientos primarios de la clase *BaseSolutionType* se muestran en la figura de la derecha (tomada de [1]).

* Observése que el programa principal ejecuta procedimientos específicos de *BaseSolutionType*.

* Obsérvese también que en este caso no se ejecuta el procedimiento TU, pues esta es tarea del *TimingModule*.

* En el procedimiento CA es donde se implementan los métodos iterativos para resolver el sistema de ecuaciones generado por los modelos e intercambios que se agreguen a la simulación.

* Cuando el sistema es no lineal, se resuelve una forma linealizada del sistema de manera iterativa usando métodos iterativos y precondicionados.

* El solucionador descrito en [3] ha sido extendido en MODFLOW 6 para incluir los aceleradores lineales CG, BiCGSTAB e ILU.

* La naturaleza no lineal de los sistemas de ecuaciones, se gestiona con los métodos *backtracking*, *pseudo-transient continuation*, *under-relaxation methods* y *Newton Dampening*.

Las siguientes figuras muestran diagramas de flujo de los métodos lineales y no lineales implementados en el procedimiento CA.

<img src="../figures/modflow03_2.png" width=250px hspace="5" vspace="5" style="float: left;"/>
<img src="../figures/modflow03_1.png" width=450px hspace="5" vspace="5" style="float: left;"/>


## Los modelos. 

**Jerarquía de clases de los modelos.**

<img src="../figures/modflow_class_2.png" width=200px hspace="5" vspace="5" style="float: left;"/>

* Todos los modelos deben ser subclases de la clase principal *BaseModelType*.

* Un modelo numérico está definido por la clase *NumericalModelType* y es un tipo especial de un modelo diseñado para colaborar con la clase *NumericalSolutionType*.

* El modelo GWF está definido en la clase *GwfModelType* que es una subclase de la clase *NumericalModelType*.

En MODFLOW 6 un modelo es una componente principal de la simulación y representa un proceso hidrológico (flujo subterráneo, flujo laminar o turbulento en conductos, flujo superficial, transporte de calor o de soluto, entre otros).

<img src="../figures/modflow04.png" width=350px hspace="5" vspace="5" style="float: left;"/>

* Los procedimientos que se ejecutan de *BaseModelType* , desde el programa principal, son los que se muestran en la figura de la izquierda (tomada de [1]).

* Estos métodos o procedimientos, generalmente están vacíos (son abstractos en términos de la POO) y pueden ser implementados por métodos que están definidos en las subclases de *BaseModelType* (sobrecarga).

* Obsérvese que en este caso no se ejecutan los procedimientos TU ni CA, pues ellos están definidos en otras componentes de la simulación.

* Esta construcción hace más fácil la adaptación y modificación de nuevos modelos.

### Modelos numéricos.

En la figura se muestran los procedimientos primarios que se ejecutan de las clases *NumericalModelType* y *NumericalSolutionType*.

<img src="../figures/modflow05.png" width=475px hspace="5" vspace="5" style="float: left;"/>

* CR: Crea el modelo numérico y los paquetes requeridos.
* DF: Define el modelo numérico leyendo la información del tamaño del modelo.
* AC: Adiciona las conexiones del modelo a Numerical Solution reservando espacios dentro de la matriz de coeficientes.
* MC: Crea un arreglo de índices que mapea las conexiones del modelo dentro del sistema de ecuaciones.
* AR: Asigna memoria para los arreglos del modelo y lee información que será constante durante toda la simulación.
* RP: Lee información del modelo de archivos, conforme es requerida, para actualizar los estreses hidrológicos u otras entradas que varían con el tiempo.
* AD: Avanza el modelo al siguiente paso en el tiempo, típicamente almacenando el valor anterior de las variables dependientes.
* CF: Calcula y actualiza los coeficientes que dependen de los resultados de la última iteración.
* FC: Calcula y agrega los términos del modelo a la matriz de coeficientes y al RHS de Numerical Solution.
* NR: Calcula y agrega los términos del método de Newton-Raphson del modelo a la matriz de coeficientes y al RHS de Numerical Solution.
* CC: Realiza una revisión de la convergencia en las variables dependientes que no son parte del modelo numérico. 
* ND: Ajusta los valores de las variables dependientes, lo que puede mejorar la convergencia para modelos que usan la formulación de Newton-Raphson.
* BD: Calcula el balance con base en la solución actualizada para la variable dependiente.
* OT: Escribe los resultados de la simulación en archivos para cada paso de tiempo o como sea requerido.
* FP: Escribe mensajes de terminación y cierra todos los archivos.
* DA: Desaloja toda la memoria.

### Groundwater Flow Model (GWF)

El modelo GWF es un tipo específico de un modelo numérico y está definido en la clase *GwfModelType* la cual está diseñada para colaborar con la clase *IterativeModelSolutionType*. La figura muestra un diagrama de flujo donde se muestran los procedimientos primarios que son ejecutados de ambas clases, veáse [4].

<img src="../figures/modflow06.png" width=500px hspace="5" vspace="5" style="float: right;"/>

* CR: crea un objeto del modelo GWF con el nombre especificado y es almacenado en una lista de modelos. También crea objetos de todos los paquetes asociados a la simulación, además asigna valores a algunos parámetros del modelo.
* DF: abre archivos, lee la información acerca del tamaño del modelo, la conectividad de las celdas de la malla y crea los objetos de los paquetes que definen las condiciones de frontera.
* AC: informa al solucionador IMS sobre el número total de celdas del modelo GWF y cómo están conectadas entre ellas; esta información es usada por IMS para configurar el tamaño y patrón de conectividad de la matriz del sistema de ecuaciones.
* MC: este procedimiento se ejecuta después de que el patrón de conectividad ha sido determinado para IMS. Se crea un arreglo que mapea la posición de las celdas del modelo GWF con sus conecciones dentro del sistema de ecuaciones. Esta información es usada por GWF que son parte de los procedimientos que agregan términos al sistema de ecuaciones.
* AR: se determinan algunos parámetros hidrológicas y se reserva memoria para los componentes de la simulación que no fueron definidas por el procedimiento DF. También se leen los datos que se mantienen constantes de un periodo de estrés a otro. Estos datos incluyen: parte de la información de las fronteras, cargas hidráulicas iniciales y propiedades hidráulicas del acuífero. Se realizan algunos cálculos preliminares para preparar los datos para un procesamiento posterior.
* RP: se ejecuta al inicio de cada paso de tiempo, pero solamente lee y procesa bloques de información si es el primer paso de tiempo del periodo de estress. Esta información incluye razones de bombeo y áreas de recarga, por ejemplo.
* AD: inicializa la carga, realiza realiza sustitución de series de tiempo y ejecuta los procedimientos de los paquetes de fronteras individuales. También realiza otros procesamientos que deben ser efectuados al principio de cada paso de tiempo.
* CF: calcula términos que serán requeridos en otros procedimientos. También rehumedece las celdas, si es necesario, y vuelve inactivas las celdas secas.
* FC: adiciona los coeficients a la matriz $A$ y al vector de lado derecho $b$.
* NR: adiciona los términos de Newton-Raphson a la matriz $A$ y al vector de lado derecho $b$. Solo se invoca cuando el método de Newton-Raphson es usado por el modelo GWF.
* CC: se realiza una verificación de la convergencia de variables que no son parte de la solución numérica, particularmente sobre paquetes avanzados que trabajan sobre la frontera.
* ND: se ejecuta el procedimiento Newton Dampening para amortiguar cambios de cargar grandes e irreales. Específicamente, si la carga simulada de una celda está por debajo del límite inferior del modelo, esta se ajusta hacia arriba, hacia el límite inferior del modelo.
* BD: calcula los términos del balance de flujo para el modelo GWF y los paquetes de frontera.
* FP: escribe la salida final del modelo y realiza algunas tareas de procesamiento final.
* DA: Desaloja toda la memoria utilizada por el modelo.

# Paquetes.

En la sección anterior se describió que el modelo GWF opera ejecutando una secuencia de procedimientos primarios, y estos son fundamentales desde el punto de vista de un programa de cómputo. No obstante, los usuarios prefieren pensar en las caracterísiticas de un programa en términos de sus capacidades para resolver problemas hidrológicos. 

Para este propósito, el modelo GWF se divide en paquetes. Un paquete es la parte del modelo que se ocupa de un único aspecto de la simulación. Por ejemplo los paquetes *River* y *Well* se encargan de todos los aspectos de los ríos y de los pozos, respectivamente. 

La siguiente figura, tomada de [4], muestra un ejemplo de configuración de GWF con varios paquetes.

<img src="../figures/gwf_mf6.png" width=450px hspace="5" vspace="5"/>

Existen tres tipos de paquetes hidrológicos y un paquete de salida. La tabla siguiente, tomada de [4], muestra la lista completa de paquetes para GWF.

<img src="../figures/gwf_mf6_pack.png" width=500px hspace="5" vspace="5"/>

* **Hydrologic/Internal**. Paquetes para simular el flujo entre celdas adyacentes o para manejar cambios de almacenamiento para todas las celdas del modelo. En general se usan para calcular los términos necesarios para resolver la ecuación de flujo de agua subterránea para cada celda del modelo o para almacenar la información necesaria para calcular estos términos.

    - **DIS**, **DISV**, **DISU**. Discretización espacial. Paquete para calcular o gestionar áreas y volúmenes de cada celda de la malla, así como las propiedades geométricas de las conexiones entre ellas.
    - **IC** (Initial Conditions). Condiciones iniciales. Paquete para leer los valores iniciales de la carga hidráulica para el modelo GWF.
    - **NPF** (Node-Property Flow). Propiedades de flujo de los nodos. Paquete para calcular la conductancia hidráulica entre celdas adyacentes, gestiona la humectación y el secado de las celdas y calcula el flujo entre celdas adyacentes.
    - **HFB** (Horizontal Flow Barrier). Barrera de flujo horizontal. Paquete complementario que, en conjunto con el paquete NPF, permite modificar las conductancias para simular una barrera horizontal entre nodos adyacentes.
    - **GNC** (Ghost Node Correction). Corrección de nodos fantasma. Paquete complementario que, en conjunto con el paquete NPF, permite mejorar la precisión de los cálculos de flujo para algunos tipos de malla.
    - **STO** (Storage). Almacenamiento. Paquete para calcular el cambio en el volumen de agua que ocurre durante un paso de tiempo.

* **Hydrologic/Stress**. Paquetes que formulan los coeficientes que describen flujos externos o en la frontera del dominio, como ríos, pozos y recarga. Por ejemplo, el paquete *River* calcula los coeficientes que describen el flujo entre una celda y un río superficial. Los paquetes de este tipo son siete: **CHD**, **WEL**, **RCH**, **RIV**, **GHB**, **DRN** y **EVT**. Estos paquetes proveen la funcionalidad que ha estado presente en versiones previas de MODFLOW.

* **Hydrologic/Advanced Stress**. Paquetes para simular estreses más avanzados. Estos paquetes incluyen **MAW**, **SFR**, **UZF** y **MVR**.

* **Output**. Paquetes que no entran en la categoría de paquetes hidrológicos:
    - **OBS** (Observation). Paquete para la observación e impresión de la salida.
    - **OC** (Output Control). Paquete para el guardado de los resultados en archivos.


# Referencias

[1] Hughes, J.D., Langevin, C.D., and Banta, E.R., 2017, Documentation for the MODFLOW 6 framework: U.S. Geological Survey Techniques and Methods, book 6, chap. A57, 40 p. https://doi.org/10.3133/tm6A57.

[2] Langevin, C. D., Hughes, J. D., Provost, A. M., Russcher, M. J., & Panday, S. (2023). MODFLOW as a configurable Multi‐Model Hydrologic Simulator. Ground Water. https://doi.org/10.1111/gwat.13351.

[3] Hughes, J.D., and White, J.T., 2013, Use of general purpose graphics processing units with MODFLOW: Groundwater, v. 51, no. 6, p. 833–846, accessed June 27, 2017. https://doi.org/10.1111/gwat.12004.

[4] Langevin, C.D., Hughes, J.D., Provost, A.M., Banta, E.R., Niswonger, R.G., and Panday, Sorab, 2017, Documentation for the MODFLOW 6 Groundwater Flow (GWF) Model: U.S. Geological Survey Techniques and Methods, book 6, chap. A55, 197 p., accessed August 4, 2017. https://doi.org/10.3133/tm6A55.